In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv('6-Beta-Round-Projects.csv')

In [2]:
# Drop uninformative features
columns_to_drop = ['projectNumber', 'createdAtBlock', 'statusUpdatedAtBlock','metadata.signature', 'metadata.application.round', 'metadata.application.project.lastUpdated',
                   'metadata.application.project.id', 'metadata.application.project.logoImg',
                   'metadata.application.project.metaPtr.protocol', 'metadata.application.project.metaPtr.pointer',
                   'metadata.application.project.bannerImg', 'metadata.application.project.credentials.twitter.@context',
                   'metadata.application.project.credentials.twitter.type', 'metadata.application.project.credentials.twitter.credentialSubject.id',
                   'metadata.application.project.credentials.twitter.credentialSubject.@context', 'metadata.application.project.credentials.twitter.credentialSubject.provider',
                   'metadata.application.project.credentials.twitter.credentialSubject.hash', 'metadata.application.project.credentials.twitter.issuer',
                   'metadata.application.project.credentials.twitter.issuanceDate', 'metadata.application.project.credentials.twitter.proof.type',
                   'metadata.application.project.credentials.twitter.proof.proofPurpose', 'metadata.application.project.credentials.twitter.proof.verificationMethod',
                   'metadata.application.project.credentials.twitter.proof.created', 'metadata.application.project.credentials.twitter.proof.jws',
                   'metadata.application.project.credentials.twitter.expirationDate', 'metadata.application.project.credentials.github.@context',
                   'metadata.application.project.credentials.github.type', 'metadata.application.project.credentials.github.credentialSubject.id',
                   'metadata.application.project.credentials.github.credentialSubject.provider', 'metadata.application.project.credentials.github.credentialSubject.hash',
                   'metadata.application.project.credentials.github.credentialSubject.@context', 'metadata.application.project.credentials.github.issuer',
                   'metadata.application.project.credentials.github.issuanceDate', 'metadata.application.project.credentials.github.proof.type',
                   'metadata.application.project.credentials.github.proof.proofPurpose', 'metadata.application.project.credentials.github.proof.verificationMethod',
                   'metadata.application.project.credentials.github.proof.created', 'metadata.application.project.credentials.github.proof.jws',
                   'metadata.application.project.credentials.github.expirationDate']
df.drop(columns=columns_to_drop, inplace=True)

In [3]:
# Separate the application questions
all_questions = df['metadata.application.answers'].apply(lambda x: [question['question'] for question in eval(x)]).explode().unique()

for question in all_questions:
    # Find the row index that contains the desired question
    question_index = df['metadata.application.answers'].apply(lambda x: any(q['question'] == question for q in eval(x)))

    # Extract the answer for the desired question from each row
    def extract_answer(row):
        try:
            return next(q['answer'] for q in eval(row) if q['question'] == question)
        except (KeyError, StopIteration):
            return None

    answers = df.loc[question_index, 'metadata.application.answers'].apply(extract_answer)

    # Create a new column with the extracted answers
    column_name = question.replace(' ', '_')  # Replace spaces with underscores for column name
    df[column_name] = pd.Series(answers)

df.drop('metadata.application.answers', axis=1, inplace=True)

In [4]:
# Combine similar questions
groups = {
    'email': ['Email_Address', 'Email_Address_(for_us_to_share_updates_and_contact_you_directly)', 'Email_'],
    'additionalInfo': ["Anything_else_you'd_like_to_share_about_your_project,_previous_work,_or_other_project_affiliations?_Anything_you'd_like_to_add_that_has_not_been_covered?",
                       'Is_there_any_other_information_you_would_like_to_share_about_your_project,_previous_work,_other_project_affiliations,_or_potential_conflicts_of_interest?_(or_anything_not_covered_elsewhere_in_this_application_that_you_feel_is_relevant_to_share)'],
    'externalFundingSources': ['Previous_project_funding_sources_(VC,_grants,_token_sales,_etc)',
                               'Previous_Funding_Sources_(VC,_Token_Sales,_etc)',
                               '_Previous_project_funding_sources_(VC,_grants,_token_sales,_etc)',
                               'Project_Funding_Sources(VC,_Token_Sales,_etc)',
                               'Funding_Sources'],
    'priorFundingUSD': ['Total_prior_funding_for_project_in_USD', 'Total_Prior_Funding_for_Project_(in_USD)'],
    'teamSize': ['Total_team_size', 'Total_Team_Size', 'Number_of_team_members', 'Team_Size'],
    'ageMonths': ['How_old_is_the_project?_(Months)', 'How_old_is_the_project_(in_months)'],
    'differentWalletAddress': ['If_you_participated_in_past_grant_rounds_using_a_different_project_payout_wallet_address,_please_share_it_here:',
                     'If_you_participated_in_past_Gitcoin_grant_rounds_using_a_different_project_payout_wallet_address,_please_share_it_here:',
                     'If_you_participated_in_past_grant_rounds_(i.e._DeSci_GR15)_using_a_different_project_payout_wallet_address,_please_share_it_here.'],
    'teamSocialProfiles': ['Profiles_or_socials_of_other_main_team_members_publicly_associated_with_project',
                           'Social_media_handles_of_other_main_team_members_publicly_associated_with_project'],
    'eligibilityAttested': ['Have_you_read_and_confirmed_your_grant_abides_by_the_Program_General_Eligibility_Policy?_https://gitcoin.notion.site/Gitcoin-Grants-Beta-Round-Eligibility-b7fca9fd3c26450b89ded11caaa6497f',
                               'Have_you_read_and_confirmed_your_grant_abides_by_the_Program_General_Eligibility_Policy?_https://gitcoin.notion.site/Gitcoin-Grants-Beta-Round-Eligibility-b7fca9fd3c26450b89ded11caaa6497f_(copy_and_paste_link_into_your_browser)',
                               'Have_you_read_and_confirmed_your_grant_abides_by_the_Program_General_Eligibility_Policy?_https://www.notion.so/tecommons/Eligibility-Policy-7cf5ff8c323149cf84011712f070f009?pvs=4',
                               'Have_you_read_and_confirmed_your_project_abides_by_the_Program_General_Eligibility_Policy?_https://gitcoin.notion.site/Gitcoin-Grants-Beta-Round-Eligibility-b7fca9fd3c26450b89ded11caaa6497f'],
    'eligibilityExplanation': ['How_does_your_project_fit_into_the_OSS_round_eligibility_criteria?_',
                         'How_does_your_project_fit_into_the_ETH_Infra_round_eligibility_criteria?',
                         'How_does_your_project_fit_into_the_ZK_Tech_round_eligibility_criteria?_',
                         'How_does_your_project_fit_into_the_Community_&_Education_round_eligibility_criteria?_',
                         'How_does_your_project_fit_into_the_GCC_round_eligibility_criteria?_',
                         'Please_describe_how_your_project_meets_the_round_eligibility_criteria._'],
    'updates': ['Please_share_any_new_updates_or_milestones_from_the_prior_season.',
                'Please_share_any_new_updates_or_milestones_from_the_prior_months',
                "If_you've_participated_in_past_grant_rounds,_please_share_any_new_updates_or_milestones_from_the_prior_months",
                'If_this_project_participated_in_past_grant_rounds,_please_share_any_new_updates_or_milestones_since_the_last_round.']}

for group, columns in groups.items():
    df[group] = df[columns].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
    df.drop(columns=columns, inplace=True)

In [5]:
# Drop questions with encryption or less than 100 answers 
df.replace('', np.nan, inplace=True)
value_counts = df.count()
columns_to_drop = value_counts[value_counts < 100].index
df.drop(columns=columns_to_drop, inplace=True)

In [6]:
# Rename columns for easier comprehension
column_names = {
    'id': 'projectId',
    'status': 'accepted',
    'uniqueContributors': 'uniqueVoters',
    'metadata.application.recipient': 'recipient',
    'metadata.application.project.createdAt': 'datetime',
    'metadata.application.project.title': 'title',
    'metadata.application.project.description': 'description',
    'metadata.application.project.website': 'website',
    'metadata.application.project.userGithub': 'userGithub',
    'metadata.application.project.projectGithub': 'projectGithub',
    'metadata.application.project.projectTwitter': 'twitter'
}
df.rename(columns=column_names, inplace=True)

In [7]:
# Add features for mean and median vote amounts
df_votes = pd.read_csv('7 beta_round_votes.csv')
mean_by_project_round = df_votes.groupby(['projectId', 'roundId'])['amountUSD'].mean()
median_by_project_round = df_votes.groupby(['projectId', 'roundId'])['amountUSD'].median()
df = df.merge(mean_by_project_round.rename('meanVoteUSD'), on=['projectId', 'roundId'], how='left')
df = df.merge(median_by_project_round.rename('medianVoteUSD'), on=['projectId', 'roundId'], how='left')

In [10]:
# Replace round ID with round name
round_stats_df = pd.read_csv('3 beta_round_stats.csv')
df['Truncated roundId'] = df['roundId'].str[:30]
round_stats_df['Truncated ID'] = round_stats_df['Round ID'].str[:30]
merged_df = pd.merge(df, round_stats_df, left_on='Truncated roundId', right_on='Truncated ID', how='left')
merged_df['roundId'] = merged_df['Round name']
merged_df.drop(['Round ID', 'Truncated ID', 'Round name', 'Truncated roundId'], axis=1, inplace=True)
df = merged_df
df.rename(columns={'roundId': 'roundName'}, inplace=True)

In [11]:
# Combine responses of no external funding
replacement = 'none'
similar_responses = ['none - self bootstrap','not funded yet','no','self funded','0','N/a',
                     "Purely self-funded using Noah Chon Lee's and Jehan Azad's savings",'Friends, self','NaN',
                     'Team members','-','no vc funding','Team funding','No previous funding','not funded',
                     'No token, 0 VC funding. Fully Self-Bootstrapped','Self-funded','Not available','nothing',
                     'Independent and bootstrapped','Team member','Self-funded through service business',
                     'Bootstrapped and volunteer','our own funds','no funding yet','my own money','Personal investment',
                     'Out of pocket expenses',"It was never financed, it's something I do for fun",
                     'No, actively looking for grants and donations', 'NO', 'None', 'No', 'NA', 'N/A',
                     'Self-funded, working full-time as a therapist while undertaking my doctorate studies.',
                     'Our startup capital comes from our team members, and we have been working for love rather than money, even without any profits yet.',
                     'own money','Nil','non we are starting','bootstrapped','No funding before',"founders' investment",'N.A.',
                     'None to date','non','zero funding','no previous funding sources','entirely own funded',
                     'Founders investments','No VC or Token Sales','Personal',"none, we're a bootstrapping startup in Africa",
                     'bootstrapping','Own savings','None yet.','Our own money','Not funded (bootstrapped)','Individual Investment',
                     'No Funding','0','/',"We are currently a self-funded organization, relying on our team's personal resources. We're actively seeking financial support to help us scale our efforts and continue delivering valuable services to the Web3 ecosystem. Thank you for your support in help us Build and Onboard to ENS and the greater Web3.",
                     'self','self raised','No VC & No Token','Bootstrap','Self and partners','No funds raised']
df['externalFundingSources'] = df['externalFundingSources'].replace(similar_responses, replacement)

In [12]:
# Convert application status and eligibility attestation to boolean
status_mapping = {'APPROVED': True, 'REJECTED': False}
df['accepted'] = df['accepted'].map(status_mapping).astype(bool)
eligibility_mapping = {'yes': True, 'no': False}
df['eligibilityAttested'] = df['eligibilityAttested'].str.lower().map(eligibility_mapping).astype(pd.BooleanDtype())

In [15]:
# Convert creation datetime to datetime data type
df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
df['datetime'] = df['datetime'].dt.floor('s')

In [16]:
# Convert prior funding amounts, team size, and age from object to float
df['priorFundingUSD'] = df['priorFundingUSD'].str.replace('[\sUSD$,~]', '', regex=True)
df['priorFundingUSD'] = df['priorFundingUSD'].str.replace('[kK]', '000', regex=True)
df['priorFundingUSD'] = df['priorFundingUSD'].astype(float)
df['teamSize'] = df['teamSize'].apply(lambda x: int(round(float(x))) if pd.notna(x) else np.nan)
df['ageMonths'] = df['ageMonths'].apply(lambda x: re.sub('\D', '', str(x)) if str(x).isdigit() else np.nan).astype(float)

In [17]:
# Add feature for number of rounds the projectId applied for
rounds_applied = df['projectId'].value_counts()
df['entries'] = df['projectId'].map(rounds_applied)

In [21]:
# Reorder the columns
new_column_order = ['projectId', 'entries', 'roundName', 'accepted', 'votes',
        'uniqueVoters', 'amountUSD', 'meanVoteUSD', 'medianVoteUSD',
        'title', 'description', 'datetime', 'recipient', 'website',
        'userGithub', 'projectGithub', 'twitter', 'ageMonths','externalFundingSources', 
        'priorFundingUSD', 'teamSize', 'teamSocialProfiles', 'eligibilityAttested',
        'eligibilityExplanation', 'updates', 'additionalInfo'] 
df = df[new_column_order]

In [25]:
print(df.dtypes)

projectId                         object
entries                            int64
roundName                         object
accepted                            bool
votes                              int64
uniqueVoters                       int64
amountUSD                        float64
meanVoteUSD                      float64
medianVoteUSD                    float64
title                             object
description                       object
datetime                  datetime64[ns]
recipient                         object
website                           object
userGithub                        object
projectGithub                     object
twitter                           object
ageMonths                        float64
externalFundingSources            object
priorFundingUSD                  float64
teamSize                         float64
teamSocialProfiles                object
eligibilityAttested              boolean
eligibilityExplanation            object
updates         

In [22]:
print(df.columns)

Index(['projectId', 'entries', 'roundName', 'accepted', 'votes',
       'uniqueVoters', 'amountUSD', 'meanVoteUSD', 'medianVoteUSD', 'title',
       'description', 'datetime', 'recipient', 'website', 'userGithub',
       'projectGithub', 'twitter', 'ageMonths', 'externalFundingSources',
       'priorFundingUSD', 'teamSize', 'teamSocialProfiles',
       'eligibilityAttested', 'eligibilityExplanation', 'updates',
       'additionalInfo'],
      dtype='object')


In [24]:
# Export to csv
csv_file_path = 'Beta Round Complete.csv'

# Export the DataFrame to CSV
df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to '{csv_file_path}' successfully.")

DataFrame exported to 'Beta Round Complete.csv' successfully.
